In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [14]:
fashion_mnist = keras.datasets.fashion_mnist

In [15]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [16]:
train_images = train_images/255.0
test_images = test_images/255.0

In [18]:
train_images[0].shape

(28, 28)

In [21]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [27]:
## Create a model
def build_model(hp):
    model = keras.Sequential([
     keras.layers.Conv2D(
        filters=hp.Int('convo_1_filter', min_value= 32,max_value = 128, step = 16),
        kernel_size= hp.Choice('convo_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28, 28, 1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('convo_2_filter', min_value= 32,max_value = 64, step = 16),
        kernel_size= hp.Choice('convo_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value = 32, max_value= 128, step = 16),
        activation='relu'
    ),
        keras.layers.Dense(10, activation='softmax')
    ])
    
     ## Compile a model     
    model.compile(optimizer = tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), # Learning Rates
                      loss= 'sparse_categorical_crossentropy',
                      metrics=['accuracy'])
    return model

from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [28]:
tuner_search = RandomSearch(build_model,
                           objective='val_accuracy',
                           max_trials=5,
                           directory='output11',
                           project_name='Mnist Fashion')

In [29]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split= 0.1)

Trial 5 Complete [00h 21m 43s]
val_accuracy: 0.8878333568572998

Best val_accuracy So Far: 0.9070000052452087
Total elapsed time: 01h 26m 08s
INFO:tensorflow:Oracle triggered exit


In [30]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        32288     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 96)                1486944   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1,523,114
Trainable params: 1,523,114
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(train_images, train_labels, epochs=5, validation_split= 0.1, initial_epochs=3)

TypeError: fit() got an unexpected keyword argument 'intial_epochs'